<a href="https://colab.research.google.com/github/Aidin12/rag-travel-itinerary/blob/main/RAG%2B%2B_Travel_Itinerary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# - **RAG++** (Retrieval-Augmented Generation) concept: Combines data retrieval from APIs with GPT generation to create a personalized output.
# - **GPT (OpenAI)**: Used to generate natural language itineraries.
# - **OpenWeatherMap API**: To retrieve weather information for the destination.
# - **Google Places API**: To retrieve information about popular places to visit.

In [ ]:


# **Step 1: Set Up Your Environment**
# Import the necessary libraries
# 🚀 We'll need these libraries to handle API requests, JSON data, and the OpenAI language model.
import requests  # To handle API requests to get travel data
import json  # To parse JSON data from the APIs
from openai import OpenAI  # To interact with GPT models
import datetime  # To handle date operations

# Install the required packages for Google Colab
# 🛠 This will set up everything you need for your code.
!pip install openai

[OPENWEATHERMAP](https://openweathermap.org/)
[google places api](https://cloud.google.com)

In [ ]:

# **Step 2: Set Up Your API Keys**
# Replace these keys with your actual keys from OpenWeatherMap and Google Places
# 🛰 API keys are essential for accessing data from APIs. Get these keys by signing up on the respective API websites.
OPENWEATHERMAP_API_KEY = "PUT YOUR OPENWEATHERMAP API KEY"  # API key for accessing OpenWeatherMap
GOOGLE_PLACES_API_KEY = "ENTER GOOGLE PLACES API HERE"  # API key for accessing Google Places
api_key = "PASTE OPENAI API KEY"  # API key for accessing OpenAI
client = OpenAI(api_key=api_key)  # Setting up OpenAI API key to authenticate requests


The Google Places API is a service provided by Google that allows developers to integrate location-based functionalities into their applications. It provides access to a wide range of data about places such as businesses, landmarks, and points of interest. Here's a breakdown of what it offers:

📌 What Can the Google Places API Do?  
Search for Places:

Perform text or keyword searches for places (e.g., "restaurants near me").
Search within a specific geographic area using coordinates.  
Retrieve Place Details:  

Access detailed information about a place, including its name, address, phone number, reviews, and photos.  
Autocomplete Search:  

Enhance user experience by providing suggestions as users type in a search box.
Geocoding and Reverse Geocoding:  

Convert addresses into geographic coordinates or vice versa.  

Nearby Places:  

Find places within a given radius of a location (e.g., "cafes within 5 km").
Place Photos:  

Fetch photos associated with a place, like images of landmarks or storefronts.
Place Reviews and Ratings:  

Access user-generated reviews and ratings for a specific place.


In [ ]:
# **Step 3: Get User Preferences**
# 👨‍✈️ Let's collect the user input to personalize the itinerary.
# We use input statements to gather user preferences such as destination, travel dates, activities, and budget.
print("Please enter your travel preferences below:")  # Prompt user to enter travel preferences
destination = input("🌍 Destination: ")  # Get the destination city from the user
dates = input("📅 Travel dates (e.g., 2024-12-01 to 2024-12-05): ")  # Get the travel dates from the user
activities = input("🏃 Preferred activities (e.g., historical, adventure, beaches): ")  # Get preferred activities
budget = input("💸 Budget per day: ")  # Get budget per day from the user

Please enter your travel preferences below:
🌍 Destination: Brussels, Be
📅 Travel dates (e.g., 2024-12-01 to 2024-12-05): 2024-12-01 to 2024-12-05
🏃 Preferred activities (e.g., historical, adventure, beaches): waffles, mussels, art, beer, views of the city, museums
💸 Budget per day: 100


In [ ]:

# **Step 4: Retrieve Data Using APIs**
# Define a function to get weather data using OpenWeatherMap API
# 🌤 The weather data will help us determine if certain activities are suitable.
def get_weather_data(city):
    # Construct the URL for the OpenWeatherMap API using the city name and API key
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={OPENWEATHERMAP_API_KEY}&units=metric"

    # Send a GET request to the API URL
    response = requests.get(url)

    # Debugging: print status code and response content
    print(f"Request URL: {url}")
    print(f"Response Status Code: {response.status_code}")
    print(f"Response Content: {response.text}")

    # Check if the response is successful (status code 200)
    if response.status_code == 200:
        # Parse the response JSON and extract necessary weather information
        weather_data = response.json()
        return {
            "description": weather_data['weather'][0]['description'],  # Weather description (e.g., "clear sky")
            "temperature": weather_data['main']['temp']  # Temperature in Celsius
        }
    else:
        # Provide detailed error message based on the status code
        if response.status_code == 401:
            print("Error: Unauthorized. Please check your API key.")
        elif response.status_code == 404:
            print(f"Error: City '{city}' not found. Please check the spelling.")
        elif response.status_code == 429:
            print("Error: Too many requests. You may have hit the rate limit for your API key.")
        else:
            print(f"Error: Unable to fetch weather data. Status Code: {response.status_code}, Response: {response.text}")
        return None


In [ ]:

# **Step 5: Get Data on Popular Places**
# Use Google Places API to get information about popular tourist attractions.
# 🗺 We'll retrieve this data and present it in the itinerary later.
def get_places_data(city, activity_type):
      # Define mappings for better results
    activity_mapping = {
        "waffles": "cafés or dessert shops",
        "scooters": "scooter rentals or bike rentals",
        "art": "art galleries or museums"
    }

    # Map the user activity input to a more appropriate query
    mapped_activity = activity_mapping.get(activity_type.lower(), activity_type)

    # Construct the URL for Google Places API using the city, activity type, and API key
    url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query={activity_type}+in+{city}&key={GOOGLE_PLACES_API_KEY}"
    # Send a GET request to the API URL
    response = requests.get(url)
    # Check if the response is successful (status code 200)
    if response.status_code == 200:
        # Parse the response JSON and extract relevant place information
        places_data = response.json()
        places = []
        for place in places_data['results'][:5]:  # Limit results to the top 5 places
            places.append({
                "name": place['name'],  # Name of the place
                "address": place['formatted_address'],  # Address of the place
                "rating": place.get('rating', 'No rating available')  # Rating of the place, if available
            })
        return places  # Return the list of places
    else:
        # Return None if the request was unsuccessful
        return None


In [ ]:

# **Step 6: Use GPT to Generate an Itinerary**
# 🧑‍💻 This is where we put it all together. We use GPT to generate a natural language itinerary using the gathered data.
def promptGPT(prompt, model, system_requirements):
    # Imagine GPT-4 is your creative writing buddy. You give it a prompt, and it comes up with a cool story idea.
    response = client.chat.completions.create(
        # The client.chat.completions.create function in the OpenAI API allows you to generate AI responses by providing a sequence of messages that simulate a conversation.
        model=model,
        messages=[
            {"role": "system", "content": system_requirements},  # Setting the personality of the assistant, like saying "You're a storyteller."
            {"role": "user", "content": prompt}  # Providing the specific question or task, like a user asking to continue the story.
        ]
    )
    # Extracting the generated text from the response
    # Think of the response like a letter from GPT with the story written inside. We just need to open it and read it.
    return response.choices[0].message.content.strip()

In [ ]:
# **Step 7: Get the Data for User Input**
# 🚀 Now we gather weather and places data using the functions we wrote earlier.
weather_info = get_weather_data(destination)  # Get weather information for the destination
places_info = get_places_data(destination, activities)  # Get information about popular places to visit


Request URL: https://api.openweathermap.org/data/2.5/weather?q=Brussels, Be&appid=86c8800f5907e1ec96365305a2e0359f&units=metric
Response Status Code: 200
Response Content: {"coord":{"lon":4.3488,"lat":50.8504},"weather":[{"id":803,"main":"Clouds","description":"broken clouds","icon":"04n"}],"base":"stations","main":{"temp":6.82,"feels_like":3.55,"temp_min":6.01,"temp_max":7.58,"pressure":1020,"humidity":87,"sea_level":1020,"grnd_level":1017},"visibility":10000,"wind":{"speed":5.14,"deg":270},"clouds":{"all":75},"dt":1732750865,"sys":{"type":1,"id":1227,"country":"BE","sunrise":1732778365,"sunset":1732808539},"timezone":3600,"id":2800866,"name":"Brussels","cod":200}


In [ ]:
# **Step 8: Generate and Print the Itinerary**
# 🗺 Now, we bring it all together and generate a final itinerary for the user.
if weather_info and places_info:
    # Generate the travel itinerary based on user preferences, weather info, and places info
    system_requirements = "You are a travel assistant who is very friendly and informative."
    prompt = f"You are a travel guide. Create a detailed 3-day travel itinerary for {destination}.\n\n"
    # Add weather information to the prompt if available
    if weather_info:
        prompt += f"The weather in {destination} is currently {weather_info['description']} with a temperature of {weather_info['temperature']} degrees Celsius.\n\n"
    # Add information about places to visit
    prompt += "Places to visit include:\n"
    for place in places_info:
        prompt += f"- {place['name']}, located at {place['address']}, has a rating of {place['rating']}\n"
    prompt += "\nPlease generate a daily plan for the traveler."

    # Use promptGPT function to generate the itinerary
    itinerary = promptGPT(prompt, "gpt-4", system_requirements)
    # Print the personalized travel itinerary
    print("\n📍 Your Personalized Travel Itinerary:\n")
    print(itinerary)
else:
    # Print an error message if any of the required data couldn't be retrieved
    if weather_info is None:
        print("Sorry, we're having trouble fetching the weather information. Please check the city name or your API key.")
    if places_info is None:
        print("Sorry, we're having trouble fetching the places information. Please check the activity type or your API key.")



📍 Your Personalized Travel Itinerary:

Day 1:
Kick start your first day in Brussels, Be.

9:00 - 10:00 AM - Breakfast at your hotel

10:30 AM - 12:30 PM - Visit the Royal Museums of Fine Arts of Belgium. Explore this home of 20,000 drawings, sculptures, and paintings, from the early 15th century to the present. 

12:30 - 2:00 PM - Have a delightful lunch at Chez Leon, located at Rue des Bouchers 18, known for its traditional Belgian cuisine.

2:30 - 5:00 PM - Head to Belgian Brewers Museum. Spend some time learning about Belgium's brewing heritage, take a self-guided tour through the museum and end your visit with a tasting of a traditional Belgian beer.

7:00 PM – Enjoy a nice dinner at Bia Mara Brussels, known for its excellent seafood dishes.

Day 2:

9:00 - 10:00 AM - Enjoy your breakfast at your hotel.

10:30 AM - 12:00 PM - Begin your day with a visit to Mount of the Arts, a cultural hub with stunning views where you can find a complex of art-related museums and buildings.

12:3

Going a bit further......
The output is more contextually aware and responsive to the user's environment.
Users get to know why certain activities are being suggested, which makes the itinerary more helpful and trustworthy.
This feature makes your itinerary more practical and realistic—ensuring the user enjoys the trip no matter the weather! 😊☀️🌧️

Key Enhancements:
Weather-Based Activity Filtering:

The function is_weather_good_for_outdoors() checks the weather description and temperature to determine if it's suitable for outdoor activities.
Dynamic Activity Selection:

When bad weather is detected, the generator filters out outdoor activities.
This is done by using a conditional check to include only indoor activities when the weather is poor.
Itinerary Output with Recommendations:

Each activity includes additional information to indicate why it was chosen:
If it's an indoor activity and the weather is bad: "Recommended due to current weather conditions".
If it's an outdoor activity and the weather is good: "Enjoyable given the current good weather conditions".
Dynamic Mapping of User Activities:

A mapping dictionary (activity_type_mapping) is used to determine if an activity is "indoor" or "outdoor".
This allows more accurate recommendations based on the weather.

In [ ]:
activity_type_mapping = {
    "waffles": "indoor",
    "scooters": "outdoor",
    "art": "indoor",
    "historical": "outdoor",
    "museums": "indoor",
    "parks": "outdoor",
    "cafes": "indoor"
}


In [ ]:
# Determine whether the weather is suitable for outdoor activities
def is_weather_good_for_outdoors(weather_description, temperature):
    bad_weather_conditions = ["rain", "thunderstorm", "snow", "drizzle"]
    # If any bad condition is mentioned in the weather description, consider it bad weather
    if any(condition in weather_description.lower() for condition in bad_weather_conditions):
        return False
    # Consider temperatures below 5°C or above 35°C as unsuitable for outdoor activities
    if temperature < 5 or temperature > 35:
        return False
    return True

# **Step 8: Generate and Print the Itinerary**
# 🗺 Now, we bring it all together and generate a final itinerary for the user.
if weather_info and places_info:
    # Determine if the weather is good for outdoor activities
    is_good_weather = is_weather_good_for_outdoors(weather_info['description'], weather_info['temperature'])

    # Generate the travel itinerary based on user preferences, weather info, and places info
    system_requirements = "You are a travel assistant who is very friendly and informative."
    prompt = f"You are a travel guide. Create a detailed 3-day travel itinerary for {destination}.\n\n"

    # Add weather information to the prompt if available
    if weather_info:
        prompt += f"The weather in {destination} is currently {weather_info['description']} with a temperature of {weather_info['temperature']} degrees Celsius.\n\n"

    # Add information about places to visit based on weather suitability
    prompt += "Places to visit include:\n"
    for place in places_info:
        activity_name = place['name'].lower()
        activity_type = activity_type_mapping.get(activity_name, "unknown")  # Check if we have the activity type in our mapping

        # Skip outdoor activities if the weather is bad
        if not is_good_weather and activity_type == "outdoor":
            continue

        # Add activity to the prompt
        prompt += f"- {place['name']}, located at {place['address']}, has a rating of {place['rating']}\n"
        # Indicate if it's recommended due to indoor/outdoor suitability
        if activity_type == "indoor" and not is_good_weather:
            prompt += "  (Recommended due to current weather conditions)\n"
        elif activity_type == "outdoor" and is_good_weather:
            prompt += "  (Enjoyable given the current good weather conditions)\n"

    prompt += "\nPlease generate a daily plan for the traveler, considering the suitability of each activity based on the weather conditions."

    # Use promptGPT function to generate the itinerary
    itinerary = promptGPT(prompt, "gpt-4", system_requirements)
    # Print the personalized travel itinerary
    print("\n📍 Your Personalized Travel Itinerary:\n")
    print(itinerary)
else:
    # Print an error message if any of the required data couldn't be retrieved
    if weather_info is None:
        print("Sorry, we're having trouble fetching the weather information. Please check the city name or your API key.")
    if places_info is None:
        print("Sorry, we're having trouble fetching the places information. Please check the activity type or your API key.")



📍 Your Personalized Travel Itinerary:

Sure, here is a sample travel itinerary for 3 days in Brussels, Belgium:

**Day 1: The Artistic Journey**
- Start your first day with a visit to the *Mount of the Arts*. Take in the beautiful cityscape views and indulge your senses in a variety of art installation or sculptures located amidst its gardens. Given the current temperature and cloudy skies, roaming around this outdoor place should be a pleasant experience.
- Right from *Mount of the Arts*, head to the *Royal Museums of Fine Arts of Belgium*. Here, you can explore a massive collection of artworks and historical artifacts. As this is in an indoor location, the weather conditions will not affect your visit.
- After a long day of walking, treat yourself to Brussels' Gourmet delight. Join the Brussels Waffle Workshop and treat your senses to the heavenly taste of freshly made Belgian waffles. Again, being an indoor activity, the weather plays no significant role.

**Day 2: Trace the Histor